In [2]:
%load_ext autoreload
%autoreload 2
import os
from cnbpy.datalad import Dataset,Subject,Abide_Subject,Abide_Dataset
from cnbpy.bids import BIDS
from cnbpy.fmriprep import FMRIPREP
import cortex

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Downloading the subject data.

Here I define an ABIDE dataset.

In [15]:
local_path='/storage/basic/nh_leverhulme/scratchpad'

Dset=Abide_Dataset(local_path)

I define a subject based on their ID number.

In [16]:
from cnbpy.datalad import Dataset,Subject,Abide_Subject,Abide_Dataset

msub=Abide_Subject(Dset,'30028')

I use datalad crawler to make a datalad dataset from their location on AWS.

In [5]:
msub.make_fmriprep_dataset()

I can now crawl through this, to download the meta data on the system.

In [6]:
msub.crawl_fmriprep_dataset()

[INFO] Loading pipeline specification from ./.datalad/crawl/crawl.cfg 
[INFO] Creating a pipeline for the fcp-indi bucket 
[WARNING] ATM we assume prefixes to correspond only to directories, adding / 
[INFO] Running pipeline [<datalad_crawler.nodes.s3.crawl_s3 object at 0x2afbbeb909e8>, switch(default=None, key='datalad_action', mapping=<<{'commit': <fu++328 chars++0>>}>>, re=False)] 
[INFO] S3 session: Connecting to the bucket fcp-indi anonymously 
[INFO] Finished running pipeline: skipped: 46 
[INFO] Total stats: skipped: 46,  Datasets crawled: 1 


Which is stored here.

In [7]:
msub.fmriprep_dset.path

'/storage/basic/nh_leverhulme/scratchpad/sub-30028/fmriprep/sub-30028'

I now download the functional data.

In [8]:
msub.make_bids()
msub.get_functional_data()

In [9]:
msub.get_sess_run_combs()

Here I see the runs and sessions for the subject.

In [10]:
msub.sess_run_combs

[['1', '1']]

## Preprocessing.

I now make a preprocessing script for the subject.

In [18]:
from cnbpy.preproc import Preprocessor
myprepoc=Preprocessor(msub)
myprepoc.make_pybest_script()

Which will exectute the following via slurm.

In [19]:
print(myprepoc.pybest_job.working_string)

#!/bin/bash

#SBATCH -J pybest_30028
#SBATCH -N 1-1
#SBATCH --cpus-per-task=4
#SBATCH --mem-per-cpu=1G
#SBATCH --mail-user=yg916972@reading.ac.uk
#SBATCH --error=/storage/basic/nh_leverhulme/pybest/pybest_30028.err
#SBATCH --output=/storage/basic/nh_leverhulme/pybest/pybest_30028.out
#SBATCH --mail-type=ALL


pybest /storage/basic/nh_leverhulme/scratchpad/sub-30028/fmriprep --space fsaverage5 --save-all --noise-source fmriprep --hemi L --out-dir /storage/basic/nh_leverhulme/pybest

pybest /storage/basic/nh_leverhulme/scratchpad/sub-30028/fmriprep --space fsaverage5 --save-all --noise-source fmriprep --hemi R --out-dir /storage/basic/nh_leverhulme/pybest



In [20]:
myprepoc.pybest_job.execute()

Now that pybest has produced the denoised data, I can now drop the original files, should I wish to.

In [ ]:
msub.drop_fmriprep_dataset()

## Analysis

I prepare the analysis for the subject, which requires the subject, a name for the analysis and an ROI to use as a source region. 
It also requires the run and session number.

In [21]:
from cnbpy.analysis import CF_analysis
myan=CF_analysis(msub,analysis_name='Test_analysis')
myan.startup(*msub.sess_run_combs[0],roilab='V1')
myan.prepare_analysis()

/home/users/yg916972/.conda/envs/p3env/lib/python3.6/site-packages/nilearn/__init__.py:69: FutureWarning: Python 3.6 support is deprecated and will be removed in release 0.10 of Nilearn. Consider switching to Python 3.8 or 3.9.
  _python_deprecation_warnings()
/home/users/yg916972/Software/prfpy/prfpy/model.py:3: UserWarning: 

 | Using Nistats with Nilearn versions >= 0.7.0 is redundant and potentially conflicting.
 | Nilearn versions 0.7.0 and up offer all the functionality of Nistats as well the latest features and fixes.
 | We strongly recommend uninstalling Nistats and using Nilearn's stats & reporting modules.

  from nistats.hemodynamic_models import spm_hrf, spm_time_derivative, spm_dispersion_derivative


Generating subsurfaces
Creating distance by distance matrices


/home/users/yg916972/.conda/envs/p3env/lib/python3.6/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:318: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)
/home/users/yg916972/.conda/envs/p3env/lib/python3.6/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:318: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)


I perform the fitting as follows.

In [22]:
myan.fit_all_folds()

  0%|          | 0/5 [00:00<?, ?it/s]/home/users/yg916972/.conda/envs/p3env/lib/python3.6/site-packages/scipy/stats/stats.py:2500: RuntimeWarning: invalid value encountered in true_divide
  return (a - mns) / sstd
/home/users/yg916972/Software/cnbpy/cnbpy/analysis.py:194: RuntimeWarning: invalid value encountered in less
  xval4weights[xval4weights<0]=0.00001
100%|██████████| 5/5 [00:07<00:00,  1.54s/it]


I summarise the fits across folds as follows.

In [23]:
myan.summarise_fits()

Which returns the fitted outcomes for each vertex.

In [24]:
myan.av_frame

,x,y,sigma,vert,R2,vert,xval_R2,session,run,roi,TRs,spliced_prf_eccentricity,spliced_prf_polar_angle
0,-0.072875,0.323436,0.500000,0.0,0.041004,0.0,0.009045,1,1,V1,151,0.331545,1.792410
1,-4.073094,-5.925365,0.500000,1.0,0.100186,1.0,0.011210,1,1,V1,151,7.190274,-2.173016
2,-5.702929,5.574198,0.528290,2.0,0.082211,2.0,0.085339,1,1,V1,151,7.974652,2.367609
3,1.629484,0.532162,0.990211,3.0,0.068832,3.0,0.006329,1,1,V1,151,1.714180,0.315663
4,-4.085898,4.346933,0.500000,4.0,0.062598,4.0,0.007859,1,1,V1,151,5.965768,2.325250
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20479,-6.372122,4.599614,0.999956,20479.0,0.071841,20479.0,0.016557,1,1,V1,151,7.858778,2.516363
20480,-1.066362,7.192217,0.513722,20480.0,0.078038,20480.0,0.052445,1,1,V1,151,7.270840,1.717990
20481,0.057322,-1.102500,0.645016,20481.0,0.122481,20481.0,0.066450,1,1,V1,151,1.103989,-1.518850
20482,0.040770,-1.572552,0.500155,20482.0,0.138875,20482.0,0.121573,1,1,V1,151,1.573081,-1.544876


I prepare the output directories for the subject.

In [25]:
myan.prepare_out_dirs()

And save out the CSV file.

In [26]:
myan.saveout()

I then define a plotter, which makes the webplotter from the CSV file.

In [68]:
from cnbpy.analysis import Plotter

In [69]:
mp=Plotter(myan)

mp.make_webplotter()

Generating new ctm file...
wm
wm
inflated
inflated


## Putting it all together

### Slurm downloading and preprocessing

Accordingly, I can download and preprocess subjects via slurm using the following functions

Essentially, I take a yaml file to populate a template script with basic information about the requested hardware etc. These are in /jobs. I then make an additional dictionary to further populate the script based on the subject I want to download.

In [14]:
%load_ext autoreload
%autoreload 2

from cnbpy.utils import send_slurm_job

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
def slurm_download_preproc(subject,local_path='/storage/basic/nh_leverhulme/scratchpad',jobout='/storage/basic/nh_leverhulme/JOBS/ABIDE'):
    job_path = pkg_resources.resource_filename('cnbpy', 'jobs')
    download_template=os.path.join(job_path,'template_Download_preproc.sh')
    download_yaml=os.path.join(job_path,'Download_preproc_config.yml')
    
    
    supdict_download={'---subno---':subject,'---local_path---':local_path}

    send_slurm_job(download_template,download_yaml,supdict_download,jobname='Download_Preproc_{subid}'.format(subid=subject),jobout=jobout)

In [11]:
slurm_download_preproc('30028')

Download_Preproc_30028 sent to SLURM


Here I download and preprocess the subject.

### Slurm model fitting and plotting

I can then perform the fitting on the subject as follows.

In [24]:
def slurm_CF_fit(subject,analysis_name,roilab,local_path='/storage/basic/nh_leverhulme/scratchpad',jobout='/storage/basic/nh_leverhulme/JOBS/ABIDE'):
    job_path = pkg_resources.resource_filename('cnbpy', 'jobs')
    fit_template=os.path.join(job_path,'template_CF_fit.sh')
    fit_yaml=os.path.join(job_path,'CF_fit_config.yml')
        
    supdict_CF_fit={'---subno---':subject,'---local_path---':local_path,'---analysis_name---':analysis_name,'---roilab---':roilab}

    send_slurm_job(fit_template,fit_yaml,supdict_CF_fit,jobname='CF_fit_{subid}_{roilab}'.format(subid=subject,roilab=roilab),jobout=jobout)

And perform all the fitting and plotting via SLURM using the following function.

In [23]:
slurm_CF_fit('30028',analysis_name='Test_analysis',roilab='V1')

CF_fit_30028 sent to SLURM


In [25]:
job_path = pkg_resources.resource_filename('cnbpy', 'jobs')

Here I perform the fitting on the subject.

Now I just need to create a bog list of subjects and submit all the jobs for each subject.